In [1]:

import os
os.makedirs('images', exist_ok=True)
os.makedirs('outputs/denoised', exist_ok=True)
os.makedirs('outputs/contrast', exist_ok=True)
os.makedirs('outputs/sharpened', exist_ok=True)
os.makedirs('outputs/final', exist_ok=True)
print("Klasörler oluşturuldu. Görselleri 'images/' klasörüne koy.")




Klasörler oluşturuldu. Görselleri 'images/' klasörüne koy.


In [2]:

import sys
!{sys.executable} -m pip install --quiet opencv-python-headless matplotlib scikit-image

import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,6)
print("Kütüphaneler yüklendi ve import edildi.")

Kütüphaneler yüklendi ve import edildi.


In [3]:
def show_images(titles, imgs, cmap='gray'):
    n = len(imgs)
    plt.figure(figsize=(4*n,4))
    for i, img in enumerate(imgs):
        plt.subplot(1,n,i+1)
        plt.imshow(img, cmap=cmap)
        plt.title(titles[i])
        plt.axis('off')
    plt.show()


In [4]:

def denoise_bilateral(img_gray, d=7, sigmaColor=75, sigmaSpace=75):
    return cv2.bilateralFilter(img_gray, d, sigmaColor, sigmaSpace)

def contrast_clahe(img_gray, clipLimit=5.0, tileGridSize=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    return clahe.apply(img_gray)

def unsharp_mask(img_gray, amount=2.5, ksize=5):
    blurred = cv2.GaussianBlur(img_gray, (ksize, ksize), 0)
    sharpened = cv2.addWeighted(img_gray, 1 + amount, blurred, -amount, 0)
    return np.clip(sharpened, 0, 255).astype(np.uint8)

def gamma_correction(img_gray, gamma=1.2):
    corrected = np.power(img_gray / 255.0, gamma) * 255
    return corrected.astype(np.uint8)

def adaptive_threshold_light(img_gray):
    thresh = cv2.adaptiveThreshold(img_gray, 255,
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((1,1), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    return cleaned



   

In [5]:
images_folder = 'images'

for img_file in os.listdir(images_folder):
    if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    img_path = os.path.join(images_folder, img_file)
    img = cv2.imread(img_path)
    if img is None:
        print(f"Hata: {img_path} okunamadı!")
        continue

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   

    # 1)Gürültü Azaltma
    den = denoise_bilateral(img_gray)
    cv2.imwrite(f'outputs/denoised/{img_file}', den)

    # 2)Kontrast
    clahe = contrast_clahe(den)
    cv2.imwrite(f'outputs/contrast/{img_file}', clahe)

    # 3)Keskinleştirme
    sharp = unsharp_mask(clahe)
    cv2.imwrite(f'outputs/sharpened/{img_file}', sharp)

    # 4) Gama Düzeltmesi
    gamma = gamma_correction(sharp)

    img_gray = cv2.convertScaleAbs(img_gray, alpha=1.3, beta=-10)

    # 5) Adaptive Threshold 
    final = adaptive_threshold_light(gamma)
    cv2.imwrite(f'outputs/final/{img_file}', final)

    show_images(['Orijinal', 'Gürültü Azaltma', 'Kontrast', 'Keskinleştirme', 'Gamma', 'Final'],
                [img_gray, den, clahe, sharp, gamma, final])
